#Comparação de Desempenho dos Processos de Negócio - Breve e PAE

Este estudo tem por objetivo identificar se houve ganho de desempenho na execução de processos administrativos a partir da adoção de um novo sistema que teve como premissa o otimização e automação dos processos de negócio de um órgão do Poder Judiciário Federal

Esta análise levou em consideração três indicadores de desempenho:

*   **Tempo total**: Tempo decorrido em dias do início ao encerramento do processo
*   **Quantidade de tramitações**: Número de tramitações entre as unidades organizacionais realizadas durante a tramitação do processo
*   **Tempo médio de tramitação**: Tempo médio em que o processo fica sendo analisado pelas unidades organizacionais.


##Importação das Bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##Carga e Preparação dos Dados do Breve

In [ ]:
dados_breve = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Dados/DS - Processo ADM/dados.breve.xlsx')

dados_breve.columns = ['ID', 'NOME_PROCESSO', 'UNIDADE_ORIGEM', 'USUARIO_ORIGEM', 'DATA_INICIO', 'DATA_FIM', 'TEMPO_TOTAL', 'QUANTIDADE_TRAMITACOES', 'TEMPO_MEDIO_TRAMITACAO', 'NOME_SISTEMA', 'ANO', 'NOME_PROCESSO_PAE']

dados_breve.describe()

,ID,TEMPO_TOTAL,QUANTIDADE_TRAMITACOES,TEMPO_MEDIO_TRAMITACAO,ANO
count,65001.000000,65001.000000,65001.000000,65001.000000,65001.000000
mean,392021.219843,18.162270,4.481639,8.228598,2010.751235
std,139595.941820,106.221643,4.111852,96.719882,2.129079
min,8283.000000,0.000000,1.000000,0.000000,2006.000000
25%,345786.000000,1.010000,2.000000,0.350000,2009.000000
50%,420531.000000,4.170000,3.000000,1.270000,2011.000000
75%,498387.000000,13.990000,4.000000,3.460000,2012.000000
max,578701.000000,3909.130000,38.000000,3909.130000,2014.000000


In [ ]:
dados_breve['NOME_PROCESSO'].unique()

array(['Solicitação de Recesso - ZE', 'Indicação de Substituto - ZE',
       'Solicitação de Recesso - TRE', 'Indicação de Substituto - TRE',
       'Indicação de Substituição de Chefe de Cartório pelo Diretor-Geral do TRESC',
       'Alteração de Férias - ZE', 'Alteração de Férias - TRE',
       'Alteração de Férias por Interesse do Servidor',
       'Cancelamento de Recesso - ZE',
       'Indenização de Transporte para Oficiais de Justiça',
       'Alteração de Férias por Necessidade de Serviço',
       'Cancelamento de Recesso - TRE', 'Numeração de Ato Normativo',
       'Numeração de Ofícios', 'Diárias ZE - Solicitação',
       'Diárias Sede - Solicitação', 'Diárias - Retificação/Cancelamento',
       'Solicitação de Carteira Funcional',
       'Solicitação de Crachá Funcional',
       'Publicação de Portaria Eletrônica',
       'Solicitação de Afastamento por Motivo de Doença',
       'Pedido de Ressarcimento de Despesas Particulares com Saúde',
       'Publicação de Edital Licita

###Tratamento de registros discrepantes que podem prejudicar a análise

In [ ]:
dados_breve = dados_breve.query('(QUANTIDADE_TRAMITACOES > 1) & (TEMPO_TOTAL > 0)')

####Variável tempo total

In [ ]:
dados_breve['TEMPO_TOTAL'].describe()

count    55830.000000
mean        15.083594
std         51.936092
min          0.010000
25%          0.990000
50%          4.860000
75%         15.090000
max       3061.730000
Name: TEMPO_TOTAL, dtype: float64

In [ ]:
Q1 = dados_breve['TEMPO_TOTAL'].quantile(.25)
Q3 = dados_breve['TEMPO_TOTAL'].quantile(.75)
IIQ = Q3-Q1
limite_inferior = Q1 - (1.5 * IIQ)
limite_superior = Q3 + (1.5 * IIQ)

selecao = (dados_breve['TEMPO_TOTAL'] >= limite_inferior) & (dados_breve['TEMPO_TOTAL'] <= limite_superior)

df = dados_breve[selecao]
df['TEMPO_TOTAL'].describe()


count    50300.000000
mean         7.287664
std          8.799349
min          0.010000
25%          0.940000
50%          3.850000
75%         10.080000
max         36.230000
Name: TEMPO_TOTAL, dtype: float64

In [ ]:
dfa = df.groupby(['NOME_PROCESSO_PAE'])['TEMPO_TOTAL'].agg(['mean', 'min', 'max']).round(3)

dfa.columns = pd.MultiIndex.from_tuples((('BREVE_TEMPO_TOTAL', 'Média'), ('BREVE_TEMPO_TOTAL', 'Min'), ('BREVE_TEMPO_TOTAL', 'Max')))

dados_breve_agrupado = dfa

dados_breve_agrupado

BREVE_TEMPO_TOTAL  ...       
                                                              Média  ...    Max
NOME_PROCESSO_PAE                                                    ...       
Afastamento de Compensação                                    2.118  ...  36.13
Afastamento de Compensação - Cancelamento                     1.154  ...  26.10
Alteração de Férias por Interesse do Servidor                 1.765  ...  32.94
Alteração de Férias por Necessidade do Serviço                2.525  ...  33.03
Diárias e Indenização de Transporte                          15.460  ...  36.22
Documento Oficial                                             4.725  ...  36.18
Indenização de Transporte para Oficial de Justiça            10.589  ...  36.23
Indicação de Substituto                                       7.147  ...  36.23
Ressarcimento de Despesa Particular com Saúde                18.479  ...  36.19
Solicitação de Carteira Funcional                            13.521  ...  36.06
Solicitação de Crachá Funcional                              14.674  ...  36.05

[11 rows x 3 columns]

####Variável Quantidade de Tramitações

In [ ]:
dados_breve['QUANTIDADE_TRAMITACOES'].describe()

count    55830.000000
mean         5.052946
std          4.167801
min          2.000000
25%          3.000000
50%          4.000000
75%          4.000000
max         38.000000
Name: QUANTIDADE_TRAMITACOES, dtype: float64

In [ ]:
Q1 = dados_breve['QUANTIDADE_TRAMITACOES'].quantile(.25)
Q3 = dados_breve['QUANTIDADE_TRAMITACOES'].quantile(.75)
IIQ = Q3-Q1
limite_inferior = Q1 - (1.5 * IIQ)
limite_superior = Q3 + (1.5 * IIQ)

selecao = (dados_breve['QUANTIDADE_TRAMITACOES'] >= limite_inferior) & (dados_breve['QUANTIDADE_TRAMITACOES'] <= limite_superior)

df = dados_breve[selecao]
df['QUANTIDADE_TRAMITACOES'].describe()


count    45038.000000
mean         3.300102
std          0.873938
min          2.000000
25%          3.000000
50%          3.000000
75%          4.000000
max          5.000000
Name: QUANTIDADE_TRAMITACOES, dtype: float64

In [ ]:
dfa = df.groupby(['NOME_PROCESSO_PAE'])['QUANTIDADE_TRAMITACOES'].agg(['mean', 'min', 'max']).round(3)

dfa.columns = pd.MultiIndex.from_tuples((('BREVE_QUANTIDADE_TRAMITACOES', 'Média'), ('BREVE_QUANTIDADE_TRAMITACOES', 'Min'), ('BREVE_QUANTIDADE_TRAMITACOES', 'Max')))

dados_breve_agrupado = dados_breve_agrupado.join(dfa)
dados_breve_agrupado

BREVE_TEMPO_TOTAL  ... BREVE_QUANTIDADE_TRAMITACOES
                                                              Média  ...                          Max
NOME_PROCESSO_PAE                                                    ...                             
Afastamento de Compensação                                    2.118  ...                            5
Afastamento de Compensação - Cancelamento                     1.154  ...                            5
Alteração de Férias por Interesse do Servidor                 1.765  ...                            5
Alteração de Férias por Necessidade do Serviço                2.525  ...                            5
Diárias e Indenização de Transporte                          15.460  ...                            5
Documento Oficial                                             4.725  ...                            5
Indenização de Transporte para Oficial de Justiça            10.589  ...                            5
Indicação de Substituto                                       7.147  ...                            5
Ressarcimento de Despesa Particular com Saúde                18.479  ...                            5
Solicitação de Carteira Funcional                            13.521  ...                            4
Solicitação de Crachá Funcional                              14.674  ...                            5

[11 rows x 6 columns]

####Variável Tempo Médio entre Tramitações

In [ ]:
dados_breve['TEMPO_MEDIO_TRAMITACAO'].describe()

count    55830.000000
mean         3.518152
std         19.031779
min          0.000000
25%          0.300000
50%          1.070000
75%          2.820000
max       1530.870000
Name: TEMPO_MEDIO_TRAMITACAO, dtype: float64

In [ ]:
Q1 = dados_breve['TEMPO_MEDIO_TRAMITACAO'].quantile(.25)
Q3 = dados_breve['TEMPO_MEDIO_TRAMITACAO'].quantile(.75)
IIQ = Q3-Q1
limite_inferior = Q1 - (1.5 * IIQ)
limite_superior = Q3 + (1.5 * IIQ)

selecao = (dados_breve['TEMPO_MEDIO_TRAMITACAO'] >= limite_inferior) & (dados_breve['TEMPO_MEDIO_TRAMITACAO'] <= limite_superior)

df = dados_breve[selecao]
df['TEMPO_MEDIO_TRAMITACAO'].describe()

count    49638.000000
mean         1.383561
std          1.471747
min          0.000000
25%          0.270000
50%          0.940000
75%          1.990000
max          6.600000
Name: TEMPO_MEDIO_TRAMITACAO, dtype: float64

In [ ]:
dfa = df.groupby(['NOME_PROCESSO_PAE'])['TEMPO_MEDIO_TRAMITACAO'].agg(['mean', 'min', 'max']).round(3)

dfa.columns = pd.MultiIndex.from_tuples((('BREVE_TEMPO_MEDIO_TRAMITACAO', 'Média'), ('BREVE_TEMPO_MEDIO_TRAMITACAO', 'Min'), ('BREVE_TEMPO_MEDIO_TRAMITACAO', 'Max')))

dados_breve_agrupado = dados_breve_agrupado.join(dfa)
dados_breve_agrupado

BREVE_TEMPO_TOTAL  ... BREVE_TEMPO_MEDIO_TRAMITACAO
                                                              Média  ...                          Max
NOME_PROCESSO_PAE                                                    ...                             
Afastamento de Compensação                                    2.118  ...                         6.57
Afastamento de Compensação - Cancelamento                     1.154  ...                         4.95
Alteração de Férias por Interesse do Servidor                 1.765  ...                         6.28
Alteração de Férias por Necessidade do Serviço                2.525  ...                         6.22
Diárias e Indenização de Transporte                          15.460  ...                         6.56
Documento Oficial                                             4.725  ...                         6.60
Indenização de Transporte para Oficial de Justiça            10.589  ...                         6.59
Indicação de Substituto                                       7.147  ...                         6.60
Ressarcimento de Despesa Particular com Saúde                18.479  ...                         6.60
Solicitação de Carteira Funcional                            13.521  ...                         5.01
Solicitação de Crachá Funcional                              14.674  ...                         6.55

[11 rows x 9 columns]

##Carga e Preparação dos dados do PAE

In [ ]:
dados_pae = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Dados/DS - Processo ADM/dados.pae.xlsx')
dados_pae.describe()

,ID,TEMPO_TOTAL,QUANTIDADE_TRAMITACOES,TEMPO_MEDIO_TRAMITACAO,ANO
count,1.624600e+04,16246.000000,16246.000000,16246.000000,16246.000000
mean,2.891880e+08,17.465526,6.143543,2.969540,2018.944725
std,1.873012e+08,59.524176,14.540520,11.922219,1.113347
min,3.202000e+04,0.000000,1.000000,0.000000,2017.000000
25%,1.118970e+08,0.970000,2.000000,0.300000,2018.000000
50%,2.872970e+08,3.870000,4.000000,1.010000,2019.000000
75%,4.156020e+08,9.800000,4.000000,2.440000,2020.000000
max,6.838920e+08,1206.790000,343.000000,750.170000,2021.000000


###Tratamento de registros discrepantes que podem prejudicar a análise

####Variável tempo total

In [ ]:
dados_pae['TEMPO_TOTAL'].describe()

count    16246.000000
mean        17.465526
std         59.524176
min          0.000000
25%          0.970000
50%          3.870000
75%          9.800000
max       1206.790000
Name: TEMPO_TOTAL, dtype: float64

In [ ]:
Q1 = dados_pae['TEMPO_TOTAL'].quantile(.25)
Q3 = dados_pae['TEMPO_TOTAL'].quantile(.75)
IIQ = Q3-Q1
limite_inferior = Q1 - (1.5 * IIQ)
limite_superior = Q3 + (1.5 * IIQ)

selecao = (dados_pae['TEMPO_TOTAL'] >= limite_inferior) & (dados_pae['TEMPO_TOTAL'] <= limite_superior)

df = dados_pae[selecao]
df['TEMPO_TOTAL'].describe()


count    14006.000000
mean         4.460630
std          5.101964
min          0.000000
25%          0.890000
50%          2.900000
75%          6.050000
max         23.040000
Name: TEMPO_TOTAL, dtype: float64

In [ ]:
dfa = df.groupby(['NOME_PROCESSO'])['TEMPO_TOTAL'].agg(['mean', 'min', 'max']).round(3)

dfa.columns = pd.MultiIndex.from_tuples((('PAE_TEMPO_TOTAL', 'Média'), ('PAE_TEMPO_TOTAL', 'Min'), ('PAE_TEMPO_TOTAL', 'Max')))

dados_pae_agrupado = dfa

dados_pae_agrupado

PAE_TEMPO_TOTAL             
                                                             Média   Min    Max
NOME_PROCESSO                                                                  
Afastamento de Compensação                                   2.198  0.00  22.11
Afastamento de Compensação - Cancelamento                    0.260  0.00   5.80
Alteração de Férias por Interesse do Servidor                1.562  0.00  21.03
Alteração de Férias por Interrupção                          3.769  0.07  20.16
Alteração de Férias por Necessidade do Serviço               3.689  0.01  21.91
Contratação                                                  7.401  0.00  22.92
Diárias e Indenização de Transporte                          9.096  0.00  23.04
Documento Oficial                                            2.796  0.00  22.99
Indenização de Transporte para Oficial de Justiça            7.895  0.00  23.00
Indicação de Substituto                                      5.370  0.00  23.04
Licença para Tratamento de Saúde ou por Motivo ...           7.224  0.00  22.97
Plano de Saúde - Alteração de Opção                          6.094  0.00  20.10
Plano de Saúde - Movimentação Cadastral                      7.822  0.10  21.81
Ressarcimento de Despesa Particular com Saúde                6.816  0.28  20.82
Solicitação de Carteira Funcional                            2.877  0.00  11.18
Solicitação de Crachá Funcional                              1.077  0.00  17.96

####Variável Quantidade de Tramitações

In [ ]:
dados_pae['QUANTIDADE_TRAMITACOES'].describe()

count    16246.000000
mean         6.143543
std         14.540520
min          1.000000
25%          2.000000
50%          4.000000
75%          4.000000
max        343.000000
Name: QUANTIDADE_TRAMITACOES, dtype: float64

In [ ]:
Q1 = dados_pae['QUANTIDADE_TRAMITACOES'].quantile(.25)
Q3 = dados_pae['QUANTIDADE_TRAMITACOES'].quantile(.75)
IIQ = Q3-Q1
limite_inferior = Q1 - (1.5 * IIQ)
limite_superior = Q3 + (1.5 * IIQ)

selecao = (dados_pae['QUANTIDADE_TRAMITACOES'] >= limite_inferior) & (dados_pae['QUANTIDADE_TRAMITACOES'] <= limite_superior)

df = dados_pae[selecao]
df['QUANTIDADE_TRAMITACOES'].describe()


count    15013.000000
mean         3.325118
std          1.169378
min          1.000000
25%          2.000000
50%          4.000000
75%          4.000000
max          7.000000
Name: QUANTIDADE_TRAMITACOES, dtype: float64

In [ ]:
dfa = df.groupby(['NOME_PROCESSO'])['QUANTIDADE_TRAMITACOES'].agg(['mean', 'min', 'max']).round(3)

dfa.columns = pd.MultiIndex.from_tuples((('PAE_QUANTIDADE_TRAMITACOES', 'Média'), ('PAE_QUANTIDADE_TRAMITACOES', 'Min'), ('PAE_QUANTIDADE_TRAMITACOES', 'Max')))

dados_pae_agrupado = dados_pae_agrupado.join(dfa)
dados_pae_agrupado

PAE_TEMPO_TOTAL  ... PAE_QUANTIDADE_TRAMITACOES
                                                             Média  ...                        Max
NOME_PROCESSO                                                       ...                           
Afastamento de Compensação                                   2.198  ...                          3
Afastamento de Compensação - Cancelamento                    0.260  ...                          1
Alteração de Férias por Interesse do Servidor                1.562  ...                          6
Alteração de Férias por Interrupção                          3.769  ...                          4
Alteração de Férias por Necessidade do Serviço               3.689  ...                          6
Contratação                                                  7.401  ...                          7
Diárias e Indenização de Transporte                          9.096  ...                          7
Documento Oficial                                            2.796  ...                          7
Indenização de Transporte para Oficial de Justiça            7.895  ...                          7
Indicação de Substituto                                      5.370  ...                          7
Licença para Tratamento de Saúde ou por Motivo ...           7.224  ...                          5
Plano de Saúde - Alteração de Opção                          6.094  ...                          1
Plano de Saúde - Movimentação Cadastral                      7.822  ...                          3
Ressarcimento de Despesa Particular com Saúde                6.816  ...                          5
Solicitação de Carteira Funcional                            2.877  ...                          1
Solicitação de Crachá Funcional                              1.077  ...                          3

[16 rows x 6 columns]

####Variável Tempo Médio entre Tramitações

In [ ]:
dados_pae['TEMPO_MEDIO_TRAMITACAO'].describe()

count    16246.000000
mean         2.969540
std         11.922219
min          0.000000
25%          0.300000
50%          1.010000
75%          2.440000
max        750.170000
Name: TEMPO_MEDIO_TRAMITACAO, dtype: float64

In [ ]:
Q1 = dados_pae['TEMPO_MEDIO_TRAMITACAO'].quantile(.25)
Q3 = dados_pae['TEMPO_MEDIO_TRAMITACAO'].quantile(.75)
IIQ = Q3-Q1
limite_inferior = Q1 - (1.5 * IIQ)
limite_superior = Q3 + (1.5 * IIQ)

selecao = (dados_pae['TEMPO_MEDIO_TRAMITACAO'] >= limite_inferior) & (dados_pae['TEMPO_MEDIO_TRAMITACAO'] <= limite_superior)

df = dados_pae[selecao]
df['TEMPO_MEDIO_TRAMITACAO'].describe()


count    14724.000000
mean         1.270752
std          1.293196
min          0.000000
25%          0.260000
50%          0.920000
75%          1.770000
max          5.640000
Name: TEMPO_MEDIO_TRAMITACAO, dtype: float64

In [ ]:
dfa = df.groupby(['NOME_PROCESSO'])['TEMPO_MEDIO_TRAMITACAO'].agg(['mean', 'min', 'max']).round(3)

dfa.columns = pd.MultiIndex.from_tuples((('PAE_TEMPO_MEDIO_TRAMITACAO', 'Média'), ('PAE_TEMPO_MEDIO_TRAMITACAO', 'Min'), ('PAE_TEMPO_MEDIO_TRAMITACAO', 'Max')))

dfa

dados_pae_agrupado = dados_pae_agrupado.join(dfa)
dados_pae_agrupado

PAE_TEMPO_TOTAL  ... PAE_TEMPO_MEDIO_TRAMITACAO
                                                             Média  ...                        Max
NOME_PROCESSO                                                       ...                           
Afastamento de Compensação                                   2.198  ...                       5.49
Afastamento de Compensação - Cancelamento                    0.260  ...                       4.84
Alteração de Férias por Interesse do Servidor                1.562  ...                       5.53
Alteração de Férias por Interrupção                          3.769  ...                       5.01
Alteração de Férias por Necessidade do Serviço               3.689  ...                       5.55
Contratação                                                  7.401  ...                       5.64
Diárias e Indenização de Transporte                          9.096  ...                       5.56
Documento Oficial                                            2.796  ...                       5.58
Indenização de Transporte para Oficial de Justiça            7.895  ...                       5.51
Indicação de Substituto                                      5.370  ...                       5.62
Licença para Tratamento de Saúde ou por Motivo ...           7.224  ...                       5.63
Plano de Saúde - Alteração de Opção                          6.094  ...                       5.07
Plano de Saúde - Movimentação Cadastral                      7.822  ...                       4.85
Ressarcimento de Despesa Particular com Saúde                6.816  ...                       5.34
Solicitação de Carteira Funcional                            2.877  ...                       0.13
Solicitação de Crachá Funcional                              1.077  ...                       5.07

[16 rows x 9 columns]

##Consolidação dos Dados para Comparação

In [ ]:
dados_consolidados = dados_pae_agrupado.join(dados_breve_agrupado)
dados_consolidados

PAE_TEMPO_TOTAL  ... BREVE_TEMPO_MEDIO_TRAMITACAO
                                                             Média  ...                          Max
NOME_PROCESSO                                                       ...                             
Afastamento de Compensação                                   2.198  ...                         6.57
Afastamento de Compensação - Cancelamento                    0.260  ...                         4.95
Alteração de Férias por Interesse do Servidor                1.562  ...                         6.28
Alteração de Férias por Interrupção                          3.769  ...                          NaN
Alteração de Férias por Necessidade do Serviço               3.689  ...                         6.22
Contratação                                                  7.401  ...                          NaN
Diárias e Indenização de Transporte                          9.096  ...                         6.56
Documento Oficial                                            2.796  ...                         6.60
Indenização de Transporte para Oficial de Justiça            7.895  ...                         6.59
Indicação de Substituto                                      5.370  ...                         6.60
Licença para Tratamento de Saúde ou por Motivo ...           7.224  ...                          NaN
Plano de Saúde - Alteração de Opção                          6.094  ...                          NaN
Plano de Saúde - Movimentação Cadastral                      7.822  ...                          NaN
Ressarcimento de Despesa Particular com Saúde                6.816  ...                         6.60
Solicitação de Carteira Funcional                            2.877  ...                         5.01
Solicitação de Crachá Funcional                              1.077  ...                         6.55

[16 rows x 18 columns]

In [ ]:
dados_consolidados.to_excel('dados.consolidados.xls')

In [ ]:
dados_consolidados_media = dados_consolidados[pd.MultiIndex.from_tuples(( ('PAE_TEMPO_TOTAL', 'Média'), ('BREVE_TEMPO_TOTAL', 'Média'), 
                                                   ('PAE_QUANTIDADE_TRAMITACOES', 'Média'), ('BREVE_QUANTIDADE_TRAMITACOES', 'Média'),
                                                   ('PAE_TEMPO_MEDIO_TRAMITACAO', 'Média'), ('BREVE_TEMPO_MEDIO_TRAMITACAO', 'Média') ))]

dados_consolidados_media.columns = pd.MultiIndex.from_tuples((('TEMPO_TOTAL', 'Média PAE'), ('TEMPO_TOTAL', 'Média Breve'),  
                                       ('QUANTIDADE_TRAMITACOES', 'Média PAE'), ('QUANTIDADE_TRAMITACOES', 'Média Breve'),
                                       ('TEMPO_MEDIO_TRAMITACAO', 'Média PAE'), ('TEMPO_MEDIO_TRAMITACAO', 'Média Breve')
                                       ))

dados_consolidados_media

TEMPO_TOTAL  ... TEMPO_MEDIO_TRAMITACAO
                                                     Média PAE  ...            Média Breve
NOME_PROCESSO                                                   ...                       
Afastamento de Compensação                               2.198  ...                  0.556
Afastamento de Compensação - Cancelamento                0.260  ...                  0.499
Alteração de Férias por Interesse do Servidor            1.562  ...                  0.471
Alteração de Férias por Interrupção                      3.769  ...                    NaN
Alteração de Férias por Necessidade do Serviço           3.689  ...                  0.638
Contratação                                              7.401  ...                    NaN
Diárias e Indenização de Transporte                      9.096  ...                  1.590
Documento Oficial                                        2.796  ...                  1.258
Indenização de Transporte para Oficial de Justiça        7.895  ...                  1.515
Indicação de Substituto                                  5.370  ...                  1.741
Licença para Tratamento de Saúde ou por Motivo ...       7.224  ...                    NaN
Plano de Saúde - Alteração de Opção                      6.094  ...                    NaN
Plano de Saúde - Movimentação Cadastral                  7.822  ...                    NaN
Ressarcimento de Despesa Particular com Saúde            6.816  ...                  3.537
Solicitação de Carteira Funcional                        2.877  ...                  3.120
Solicitação de Crachá Funcional                          1.077  ...                  2.333

[16 rows x 6 columns]

In [ ]:
dados_consolidados_media.to_excel('dados.consolidados.media.xls')

In [ ]:
#converte as colunas com indexação multipla para o padrão 
dados_comp_grafico = dados_consolidados_media.copy()
dados_comp_grafico.columns = ['_'.join(col) for col in dados_comp_grafico.columns.values]

dados_comp_grafico.reset_index(inplace=True)
dados_comp_grafico.head()

,NOME_PROCESSO,TEMPO_TOTAL_Média PAE,TEMPO_TOTAL_Média Breve,QUANTIDADE_TRAMITACOES_Média PAE,QUANTIDADE_TRAMITACOES_Média Breve,TEMPO_MEDIO_TRAMITACAO_Média PAE,TEMPO_MEDIO_TRAMITACAO_Média Breve
0,Afastamento de Compensação,2.198,2.118,1.981,3.137,0.591,0.556
1,Afastamento de Compensação - Cancelamento,0.260,1.154,1.000,2.090,0.194,0.499
2,Alteração de Férias por Interesse do Servidor,1.562,1.765,2.135,3.274,0.548,0.471
3,Alteração de Férias por Interrupção,3.769,NaN,2.026,NaN,1.345,NaN
4,Alteração de Férias por Necessidade do Serviço,3.689,2.525,2.081,3.426,1.268,0.638


##Análise do Estudo Realizado 

Ficou evidente na análise um importante ganho de desempenho ao se empregar um engine de workflow (automação de processos) em consideração ao modelo anterior que grande parte das atividades eram feitas manualmente.

A integração entre os sistemas legados, otimização dos processos resultou em um ganho médio de **60%** de redução do tempo total para execução de processos administrativos de mesma natureza.